# 2025 CITS4012 Group XX Assignment

*Make sure you change the file name with your student id.*

# 0. Readme
这是Project 1的模板，我直接拿来用了

仅需注重代码风格及可视化，让marker改起来舒服就行，分数低不了的
* 例如：多抄Lab的技术栈，多引用学界的基石论文，marker自然乐意审自己熟悉的技术。e.g., WordCloud Visualization

*If there is something to be noted for the marker, please mention here.*

*If you are planning to implement a program with Object *Oriented* Programming style, please put those the bottom of this ipynb file*

## 0.1 File Structure

```
/content/drive/MyDrive/path_to_your_folder
├── CITS4012_YourGroupID.ipynb
├── sample
├── sample
├── sample
├── sample
├── sample
├── sample
├── test.json
├── train.json
└── validation.json
```

## 0.2 Setup

### Step 1 - Check current Python environment

In [ ]:
import sys
print(sys.executable)

### Step 2 - Install dependencies

> NOTE:
>
> In Google Drive, an ERROR may occur due to incompatibility of the latest versions of `numpy` and `scipy`.
>
> Simply **restart the runtime** to use the newly downgraded versions.
>
> (In other words, run this code snippet twice to make sure all requirements are satisfied.)

In [ ]:
%pip install word2number
%pip install contractions
%pip install nltk
%pip install pandas
%pip install gensim

### Step 3 - Mount source files

This step assumes you are running in a Google Colab environment.

In [ ]:
# Define your Google Drive folder path
my_folder = "MyDrive/Colab Notebooks/path_to_your_folder"

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Path setups
import os
folder_path = "/content/drive/" + my_folder

test_file = os.path.join(folder_path, "test.json")
train_file = os.path.join(folder_path, "train.json")
val_file = os.path.join(folder_path, "validation.json")

# 1. Overview

We implemented three substantially different model architectures:

* [The vanilla RNN-based encoder-decoder](#scrollTo=67TQJgOJ_lF1)

* [The Bi-LSTM encoder-decoder with XXX-attention in different positions](#scrollTo=BhSE5ON4_r0C)

* [The vanilla Transformer with self-attention (no RNN)](#scrollTo=E4w_n2P2_xxm)

# 2. Dataset Preprocessing

## 2.1 Load JSON files



In [ ]:
import pandas as pd

# Working dataframes
train_df = pd.read_json(train_file)
val_df = pd.read_json(val_file)
test_df = pd.read_json(test_file)

# Keep original copies
source_train_df = train_df.copy()
source_val_df = val_df.copy()
source_test_df = test_df.copy()

# Sneak peek
pd.set_option("display.max_colwidth", 30)

print(train_df.head())
# print(val_df.head())
# print(test_df.head())

print("Train size:", len(train_df))
print("Val size:", len(val_df))
print("Test size:", len(test_df))

pd.reset_option("display.max_colwidth")

## 2.2 Data cleansing

原则上，我们只知道训练集数据，不该为验证集/测试集的特点设计数据清洗的代码

TODO

1. 洗 146/427， 147
2. 未来处理Noise2时的抉择:
    * 拆段成句，打乱原始下标
    * 干扰Padding效率，因为每个batch的所有sent都得补齐到max_sent_len

Assumption
* The value of `label` is binary, either "neutral" or "entails".

Compromises
* Ignore syntactic errors and semantic errors.
* Apply unified premises rules on hypothesis.

Handled Issues
| No. | Description | Examples |
|----------|-------------|----------|
| Issue 1  | HTML/XML tags with ID pattern | train premise 78, 270, 319, ... |
| Issue 2  | Non-linguistic long separators | train premise 1, 382, 697, ... |
| Issue 3  | Library Catalog ID | train premise 147, 270, 319, ... |
| Issue 4  | Duplicate consecutive phrases | train premise 78, 87, 564, ... |
| Issue 5  | Duplicated whitespaces | train premise 123, 193, 259, ... |
| Issue 6  | Spaces before punctuations, except '!' and '?' | train premise 3, 333, 6280 |

Kept Noises

| No. | Description | Noise | Non-Noise |
|----------|-------------|----------|--------------|
| Noise 1  | Instructional prompt words | train premise 3, 61, 319, ... | train premise 16, 24, 61, ... |
| Noise 2  | Concatenated sentences | train premise 270, 537, 608, ... |  |
| Noise 3  | Numbered markers | train premise 270, 537, 608, ... | train premise 1546, 2068, ... |
| Noise 4  | Misplaced `label` values | train premise 270, 537, 606, ... | train premise 1683, 2068, ... |

Limitation

The difference viewer automatically escapes HTML entities, which might overlook HTML noise. (E.g., &amp;quot; in train premise 420)

In [ ]:
import re, html
from difflib import SequenceMatcher
from IPython.display import display, HTML

def cleanse(df):
  df = df.copy()

  ID_PATTERN = r"\b[A-Za-z]?(?:\d{6,}|[A-Za-z0-9]{8,})(?:-[A-Za-z0-9]{2,})+\b"
  REPEAT_PATTERN = r"\b((?:\w+\s+){0,2}\w+)( \1\b)+"

  for col in ["premise", "hypothesis"]:

    # issue 1: HTML/XML tags with ID pattern
    df[col] = df[col].apply(html.unescape)
    df[col] = df[col].apply(lambda x: re.sub(r"<[^>]*>", " ", x))
    df[col] = df[col].apply(lambda x: re.sub(ID_PATTERN, "[id]", x))

    # issue 2: non-linguistic long separators
    df[col] = df[col].apply(lambda x: re.sub(r"[-=*_~$]{3,}", " ", x))

    # issue 3: duplicate continuout words
    df[col] = df[col].apply(lambda x: re.sub(REPEAT_PATTERN, r"\1", x))

    # issue 4: duplicate whitespaces
    df[col] = df[col].apply(lambda x: re.sub(r"\s+", " ", x).strip())

    # issue 5: spaces before punctuations
    df[col] = df[col].apply(lambda x: re.sub(r"\s+([.,;:])", r"\1", x))

  return df

def verify_print(source_df, cleaned_df, tag, num, col, index, kept=False):
  status = "Kept " if kept else "Fixed"
  print(f"{tag} {num}:\t{source_df[col][index]}")
  print(f"{status} {num}:\t{cleaned_df[col][index]}")
  print()

# You can enable dark mode by replacing default "light" here
def verify_diff(source_df, cleaned_df, tag, num, col, index, kept=False, theme="dark"):
    if theme == "light":
      bg = "#fafafa"; fg = "#000000"
      del_bg = "#ffdddd"; ins_bg = "#ddffdd"
      del_fg = "#aa0000"; ins_fg = "#006600"
    else:
      bg = "#1e1e1e"; fg = "#e0e0e0"
      del_bg = "#662222"; ins_bg = "#224422"
      del_fg = "#ff9999"; ins_fg = "#99ff99"

    status = "Kept" if kept else "Fixed"

    original = source_df[col][index]
    cleaned = cleaned_df[col][index]

    matcher = SequenceMatcher(None, original, cleaned)

    html_original = f"""
    <div style='margin-bottom: 10px; background-color:{bg}; color:{fg}; white-space: nowrap; font-family: monospace; padding: 5px; border-radius:4px;'>
    <b>{tag} {num}:</b> """
    html_cleaned = f"""
    <div style='margin-bottom: 10px; background-color:{bg}; color:{fg}; white-space: nowrap; font-family: monospace; padding: 5px; border-radius:4px;'>
    <b>{status} {num}:</b> """

    for tag, i1, i2, j1, j2 in matcher.get_opcodes():
      orig_text = original[i1:i2].replace(" ", "&nbsp;")
      clean_text = cleaned[j1:j2].replace(" ", "&nbsp;")

      if tag == 'equal':
        html_original += orig_text
        html_cleaned += clean_text
      elif tag == 'delete':
        html_original += f"<span style='background-color:{del_bg}; color:{del_fg};'>{orig_text}</span>"
      elif tag == 'insert':
        html_cleaned += f"<span style='background-color:{ins_bg}; color:{ins_fg};'>{clean_text}</span>"
      elif tag == 'replace':
        html_original += f"<span style='background-color:{del_bg}; color:{del_fg};'>{orig_text}</span>"
        html_cleaned += f"<span style='background-color:{ins_bg}; color:{ins_fg};'>{clean_text}</span>"

    html_original += "</div>"
    html_cleaned += "</div>"

    display(HTML(html_original + html_cleaned + "<hr/>"))

# Clenasing
train_df = cleanse(train_df)
val_df = cleanse(val_df)
test_df = cleanse(test_df)

# Keep cleaned copies
cleaned_train_df = train_df.copy()
cleaned_val_df = val_df.copy()
cleaned_test_df = test_df.copy()

# Verification
verify_diff(source_train_df, cleaned_train_df, tag="Issue", num=1, col="premise", index=319)
verify_diff(source_train_df, cleaned_train_df, tag="Issue", num=2, col="premise", index=1)
verify_diff(source_train_df, cleaned_train_df, tag="Issue", num=3, col="premise", index=87)
verify_diff(source_train_df, cleaned_train_df, tag="Issue", num=4, col="premise", index=123)
verify_diff(source_train_df, cleaned_train_df, tag="Issue", num=5, col="premise", index=3)
verify_diff(source_train_df, cleaned_train_df, tag="Issue", num=5, col="premise", index=6280, kept=True)
verify_diff(source_train_df, cleaned_train_df, tag="Issue", num=5, col="premise", index=333, kept=True)

verify_diff(source_train_df, cleaned_train_df, tag="Issue", num="Hybrid", col="premise", index=270)

## 2.3 Normalization

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('punkt_tab')
nltk.download('stopwords')

In [ ]:
import re, contractions
from nltk.tokenize import word_tokenize, sent_tokenize
from word2number import w2n
from contractions import fix as expand_contractions

sww = stopwords.words('english')

def word2num(text):
  def convert(match):
    word = match.group(0)
    try:
      return f" {w2n.word_to_num(word)} "
    except:
      return f" {word} "
  text = re.sub(
    r'\b(?:zero|one|two|three|four|five|six|seven|eight|nine|ten|'
    r'eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen|'
    r'eighteen|nineteen|twenty|thirty|forty|fifty|sixty|seventy|'
    r'eighty|ninety|hundred|thousand|million|billion|and|[- ])+\b',
    convert, text)
  return ' '.join(text.split())

def normalize(df):
  for col in ["premise", "hypothesis"]:

    # rule 1: lowercase
    df[col] = df[col].str.lower()

    # rule 2: expand contraction
    df[col] = df[col].apply(expand_contractions)

    # rule -1: remove stopwords
    # df[col] = df[col].apply(lambda x: " ".join([w for w in word_tokenize(x) if w.lower() not in sww]))

    # rule -2: remove bracketed content
    # df[col] = df[col].apply(lambda x: re.sub(r'\([^)]*\)', '', x))

    # rule -3: symbolize linguistic numbers
    # df[col] = df[col].apply(word2num)

  return df

# Normalization
train_df = normalize(train_df)

# Verification
verify_diff(cleaned_train_df, train_df, tag="Rule", num="1", col="premise", index=1)
verify_diff(cleaned_train_df, train_df, tag="Rule", num="2", col="premise", index=420)
verify_diff(cleaned_train_df, train_df, tag="Rule", num="2", col="hypothesis", index=8, kept=True)

# Inactivate rules
# verify_diff(cleaned_train_df, train_df, tag="Rule", num=-1, col="premise", index=1, kept=True)
# verify_diff(cleaned_train_df, train_df, tag="Rule", num=-2, col="premise", index=8, kept=True)
# verify_diff(cleaned_train_df, train_df, tag="Rule", num=-3, col="premise", index=147, kept=True)

## 2.4 Tokenization

# 3. Word Embedding Construction
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

# 4. Model Implementation

## 4.1 The vanilla RNN-based encoder-decoder

## 4.2 The Bi-LSTM encoder-decoder with XXX-attention in different positions

## 4.3 The vanilla Transformer with self-attention

# 5. Performance Evaluation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

# 6. Interactive Inference Colab Form
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)